# Pipeline

In [4]:
# Importing other necessary packages
import glob
import pandas as pd
import numpy as np
import cv2 # image transformation
import os
import re
import concurrent # for parallel instances
import functools # for creating partial functions
import shutil
from pathlib import Path

from io import StringIO # to convert string to csv
import time # to measure time

# to add the path where to search for modules
import sys
sys.path.append('/home/hennes/Internship/table_scanner')

# Importing table_ocr modules 
from table_ocr import pdf_to_images
from table_ocr import extract_tables
from table_ocr import extract_cells
from table_ocr import ocr_image
from table_ocr import ocr_to_csv

## Choosing files (Choose one of the following options)

In [5]:
## setting folders and pdfs
folder = "/home/hennes/Internship/pdfs" # should be folder containing pdfs of election
save_folder = '/home/hennes/Internship/constituencies/' # folder into which csvs should be saved
old = '/home/hennes/Internship/old_files/' # folder into which old files are moved
allpdf = [pdf for pdf in glob.glob(folder+'/*') if pdf.endswith(".pdf")] # list with all pdfs from folder

### all pdfs that do not have a corresponding file in output folder

In [10]:
# exclude pdfs for which there is already a csv in save folder
pdflist = sorted([pdf for pdf in allpdf if pdf.split('/')[-1].split('_')[0].split('.')[0] not in
           [file.split('/')[-1].split('.')[0] for file in glob.glob(save_folder+'*')]])

### pdfs as per excel sheet

In [12]:
df = pd.read_excel('/home/hennes/Downloads/Book(1).xlsx')

# get relevant pdf numbers

worklist = df[df['Error Code'] == 'OCR']['Constituency number'].tolist()

# give appropriate filename endings to items

for idx, item in enumerate(worklist):
    if len(str(item)) == 1:
        worklist[idx] = f'PC00{item}.pdf'
    if len(str(item)) == 2:
        worklist[idx] = f'PC0{item}.pdf'
    if len(str(item)) == 3:
        worklist[idx] = f'PC{item}.pdf'
        
worklist = tuple(worklist)
pdflist = [pdf for pdf in allpdf if pdf.endswith(worklist)]

FileNotFoundError: [Errno 2] No such file or directory: '/home/hennes/Downloads/Book(1).xlsx'

## Running Program

In [9]:
move_files(folder, old)

For when input is image files from old folder:

In [12]:
for x in pdflist:
    imglist = [img for img in next(os.walk(old))[2] if img.startswith(f'{x}')]
    [shutil.copy(old+x, folder) for x in imglist]
    pdf = f'{folder}/{x}.pdf'
    print(f' \nWORKING ON {x}\n ')
    stop = ocr_pipeline(pdf, preprocess = False, dilate=None)
    if stop:
        break

 
WORKING ON PC003
 
created images of PC003.pdf
extracted tables of PC003.pdf
extracted cells of PC003.pdf
completed ocr of PC003.pdf
working on PC003-01
working on PC003-02
working on PC003-03
working on PC003-04
working on PC003-05
working on PC003-06
working on PC003-07
working on PC003-08
working on PC003-09
working on PC003-10
working on PC003-11
working on PC003-12
working on PC003-13
working on PC003-14
working on PC003-15
working on PC003-16
working on PC003-17
working on PC003-18
working on PC003-19
working on PC003-20
working on PC003-21
working on PC003-22
working on PC003-23
working on PC003-24
working on PC003-25
working on PC003-26
working on PC003-27
working on PC003-28
working on PC003-29
working on PC003-30
working on PC003-31
working on PC003-32
working on PC003-33
working on PC003-34
working on PC003-35
working on PC003-36
working on PC003-37
working on PC003-38
working on PC003-39
working on PC003-40
working on PC003-41
working on PC003-42
working on PC003-43
worki

For when input files are pdf:

In [21]:
for x in pdflist[10:]:
    print(f' \nWORKING ON {x}\n ')
    stop = ocr_pipeline(x)
    if stop:
        break

 
WORKING ON /home/hennes/Internship/pdfs/PC027.pdf
 
created images of PC027.pdf
preprocessed images of PC027.pdf
extracted tables of PC027.pdf
extracted cells of PC027.pdf
completed ocr of PC027.pdf
working on PC027-01
working on PC027-02
Error tokenizing data. C error: Expected 17 fields in line 4, saw 18

will try again ignoring one more line.
working on PC027-01
working on PC027-02
working on PC027-03
working on PC027-04
working on PC027-05
working on PC027-06
working on PC027-07
working on PC027-08
working on PC027-09
working on PC027-10
working on PC027-11
working on PC027-12
working on PC027-13
working on PC027-14
working on PC027-15
working on PC027-16
working on PC027-17
working on PC027-18
working on PC027-19
working on PC027-20
working on PC027-21
working on PC027-22
working on PC027-23
working on PC027-24
working on PC027-25
working on PC027-26
working on PC027-27
working on PC027-28
working on PC027-29
working on PC027-30
working on PC027-31
working on PC027-32
working on

working on PC031-42
working on PC031-43
Error tokenizing data. C error: Expected 4 fields in line 4, saw 13

will try again ignoring one more line.
working on PC031-01
working on PC031-02
working on PC031-03
working on PC031-04
working on PC031-05
working on PC031-06
working on PC031-07
working on PC031-08
working on PC031-09
working on PC031-10
working on PC031-11
working on PC031-12
working on PC031-13
working on PC031-14
working on PC031-15
working on PC031-16
working on PC031-17
working on PC031-18
working on PC031-19
working on PC031-20
working on PC031-21
working on PC031-22
working on PC031-23
working on PC031-24
working on PC031-25
working on PC031-26
working on PC031-27
working on PC031-28
working on PC031-29
working on PC031-30
working on PC031-31
working on PC031-32
working on PC031-33
working on PC031-34
working on PC031-35
working on PC031-36
working on PC031-37
working on PC031-38
working on PC031-39
working on PC031-40
working on PC031-41
working on PC031-42
working on P

working on PC037-24
working on PC037-25
working on PC037-26
working on PC037-27
working on PC037-28
working on PC037-29
working on PC037-30
working on PC037-31
working on PC037-32
working on PC037-33
working on PC037-34
working on PC037-35
working on PC037-36
working on PC037-37
working on PC037-38
working on PC037-39
working on PC037-40
working on PC037-41
working on PC037-42
working on PC037-43
working on PC037-44
working on PC037-45
working on PC037-46
working on PC037-47
working on PC037-48
working on PC037-49
working on PC037-50
working on PC037-51
working on PC037-52
working on PC037-53
working on PC037-54
working on PC037-55
working on PC037-56
working on PC037-57
working on PC037-58
working on PC037-59
working on PC037-60
working on PC037-61
working on PC037-62
working on PC037-63
working on PC037-64
working on PC037-65
working on PC037-66
working on PC037-67
working on PC037-68
working on PC037-69
working on PC037-70
working on PC037-71
working on PC037-72
working on PC037-73


## Defining Functions

In [7]:
def move_files(folder, old):
    # if folder already exist, delete it. Then move folders to old directory.
    [shutil.rmtree(old+'/'+e)
     for e in next(os.walk(folder))[1] if Path(old+'/'+e).is_dir()]

    [shutil.move(folder+'/'+e, old)
     for e in next(os.walk(folder))[1] if not e.endswith('.pdf')]

    # if files already exist, delete them. Then move files to old directory.
    [os.remove(old+'/'+e)
     for e in next(os.walk(folder))[2] if not e.endswith('.pdf') and Path(old+'/'+e).is_file()]

    [shutil.move(e, old)
     for e in glob.glob(folder+'/*') if not e.endswith('.pdf')]

In [20]:
# Importing images
def ocr_pipeline(pdf, thresh=None, no_noise=None, preprocess=True, dilate=True, image_conversion=True):
    '''Function which binds together the functions necessary to turn one pdf of several pages of tables
    into a csv file which will be stored under the same name in the specified folder.
    The options are:
    
    image_conversion - whether images should be converted from pdf. If value is None, then converted 
                       images should already be supplied in the folder.
    
    thresh           - whether otsu thresholding should be applied to cell images before performing ocr.
                       Useful if the images contain a lot of grey, in which case not thresholding likely
                       results in many artifacts wrongly identified as numbers.
    
    no_noise         - whether noise reduction techniques should be used before ocr. Should be avoided in
                       images with very thin/small font.
                
    preprocess       - whether preprocessing should be used before table extraction. Useful to avoid if
                       preprocessing results in wrongly rotated images.
                 
    dilate           - whether image should be dilated before cell extraction. Useful for thin/frail cell
                       lines, but results in numbers becoming so thick that they are recognised as cell 
                       walls themselves in images with tightly written fonts.'''
    
    stop = None
    
    if image_conversion == True:
        # extract pages from pdf and save as images 
        pdf_to_images.pdf_to_images(pdf)
        print(f"created images of {pdf.split('/')[-1]}")
        
    # define list of images thus created
    imglist = [img for img in glob.glob(folder+'/*') if img.endswith('.png')]
    
    if preprocess == True:
        # rotate and correct images for skew
        with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
            executor.map(pdf_to_images.preprocess_img, imglist)
        print(f"preprocessed images of {pdf.split('/')[-1]}")

    # define imglist as list of lists because next function needs list as argument
    imglist = [[img] for img in imglist]

    # crop images to table and save in new folder
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        executor.map(extract_tables.main, imglist)
    print(f"extracted tables of {pdf.split('/')[-1]}")

    # define list of all images of tables in newly created subfolders
    dirlist = sorted([directory for directory in glob.glob(folder+'/*/*') if directory.endswith('.png')])
    
    # If one of the images is smaller than 50 KB, table extraction probably did not work.
    # In that case, stop and continue with next pdf.
    
    if any(e for e in dirlist if os.path.getsize(e)/1000 < 50):
        print(f'problem tables: {[e.split("/")[-2] for e in dirlist if os.path.getsize(e)/1000 < 50]}')
        print(f'Table extraction did not work correctly. Continuing with next pdf.')
        move_files(folder, old)
    
    else:
        
        # Create partial function for cell extraction in which dilation is specified
        p_extract_cells = functools.partial(extract_cells.main, dilate)
        
        # Extract individual cell images
        with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
            executor.map(p_extract_cells, dirlist)
        print(f"extracted cells of {pdf.split('/')[-1]}")

        # define list of directories containing cell images
        dirlist = [directory for directory in glob.glob(folder+'/*/*/')]

        # define list of images of cells within directories
        celllists = [glob.glob(cellfolder+'*') for cellfolder in dirlist]

        # Specify that there should be no multiple threads.
        # This is important because I am already using multiple processors.
        # And create partial function to pre-specify thresh and no_noise options.
        os.environ['OMP_THREAD_LIMIT'] = '1'
        p_ocr_image = functools.partial(ocr_image.main, None, thresh, no_noise)

        # perform OCR on each image
        for image_list in celllists:
            with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
                executor.map(p_ocr_image, image_list)
        print(f"completed ocr of {pdf.split('/')[-1]}")

        try:
            gathered_data = []
            # get the names for the individual pages
            pages = sorted([filename for directory, filename in
                     [os.path.split(x) for x in glob.glob(folder+'/*') if not x.endswith(('.pdf', '.png'))]])

            # create list of alphabetically ordered lists of ocred files
            ocrlists = [sorted(y) for y in
                        [glob.glob(f'{folder}/{x}/cells/ocr_data/*.txt') for x in pages]]
            zippie = zip(pages, ocrlists)

            # for each pair of page and ocred cells, create a csv
            for y, x in zippie:
                output = ocr_to_csv.main(x)
                csv = StringIO(output)
                print(f'working on {y}')

                # Turning csv into dataframe
                # Skipping the first two rows because they have fewer columns than rest
                # Also useful for chaining of tables later
                df = pd.read_csv(csv,  header = None, skiprows=[0, 1])
                gathered_data.append(df)
                df = pd.concat(gathered_data)

                # give df a name and save it
                constituency_name = pages[0][0:5]
                df.to_csv(save_folder+constituency_name+'.csv')

            print(f'Saved {constituency_name} to folder.')

            # move old files and folders into old_files folder
            move_files(folder, old)

        # If there is an error, print error message.
        # Most likely eror is that header lines (which are not turned into cells properly)
        # are three instead of two rows. So we try reading the csvs again, this
        # time ignoring the first three instead of just two rows.
        except Exception as e:
            print(e)
            print('will try again ignoring one more line.')
                # get the names for the individual pages

            try:
                gathered_data = []
                pages = sorted([filename for directory, filename in
                         [os.path.split(x) for x in glob.glob(folder+'/*') if not x.endswith(('.pdf', '.png'))]])

                # create list of alphabetically ordered lists of ocred files

                ocrlists = [sorted(y) for y in
                            [glob.glob(f'{folder}/{x}/cells/ocr_data/*.txt') for x in pages]]
                zippie = zip(pages, ocrlists)

                # for each pair of directory and files, create csv file
                for y, x in zippie:
                    output = ocr_to_csv.main(x)
                    csv = StringIO(output)
                    print(f'working on {y}')

                    # Turning csv files into single dataframe
                    # Skipping the first two rows because they have fewer columns than rest
                    # Also useful for chaining of tables later
                    col_names = [str(e) for e in list(range(0,25))]
                    df = pd.read_csv(csv,  header = None, skiprows=[0, 1], names=col_names)
                    gathered_data.append(df)
                    df = pd.concat(gathered_data)

                    # give df a name and save it
                    constituency_name = pages[0][0:5]
                    df.to_csv(save_folder+constituency_name+'.csv')

                print(f'Saved {constituency_name} to folder.')

                move_files(folder, old)

            # If this still does not work, stop the program and try to manually correct mistakes.
            except Exception as e:
                print(e)
                print(f'There is a problem with {pdf.split("/")[-1]}. Continuing with next pdf.')
                # move old files and folders into old_files folder
                move_files(folder, old)